# Erlang B calculations
## source of truth: http://www.erlang.com/calculator/erlb/


In [182]:
from math import factorial
import pandas as pd
import math

VERBOSE = False

## Return probability of blocking given concurrency and number of lines

In [193]:
########################################################
## Calculate the probability of blocking under Erlang-B model assumptions
########################################################
def ErlangB (concurrency, num_lines):
    concurrency =float (concurrency)
    InvB = 1.0
    for j in range(1, num_lines+1):
        InvB = 1.0 + InvB * (j/concurrency)
    return (1.0 / InvB)
  

In [195]:
## Testing:
ErlangB(21, 35)

0.0013932985544363936

## Return Min Number of lines needed to guarantee ErlangB() returns less than pBlock

In [206]:
def numLines (concurrency, pBlock, abs_min_lines = None, incrementer = 1):
   
    concurrency =float (concurrency)
    if abs_min_lines is None:
        abs_min_lines = int (concurrency) + incrementer
    pB_absminlines = ErlangB(concurrency, abs_min_lines)
    pb_minlines =  ErlangB(concurrency, abs_min_lines)
    
    myDF = pd.DataFrame.from_dict({'concur': [concurrency],
                                       'pBlock': [pBlock],
                                       'lines': [abs_min_lines],
                                       'pB': [pb_minlines] }
                                      )
    #print myDF
   
    min_lines = int (abs_min_lines * 1.1)

    ###############################################################
    ##  Loop through the possible min_lines (until calculated pB becomes less then pBlock,
    ##  incrementing them by incrementer
    ###############################################################
    while pb_minlines >= pBlock:
        pb_minlines = ErlangB(concurrency, min_lines)
        mydf = pd.DataFrame.from_dict({'concur': [concurrency],
                                       'pBlock': [pBlock],
                                       'lines': [min_lines],
                                       'pB': [pb_minlines] }
                                      )
        ##print mydf     
        myDF = myDF.append(mydf)
        min_lines = int (min_lines + incrementer)
    
    if VERBOSE:
        return myDF
    else:
        return mydf

In [207]:
numLines (100, 0.001)

,concur,lines,pB,pBlock
0,100,128,0.000968,0.001


In [205]:
## Testing:
myDF = None
for cc in (100, 110, 120):
    for pp in (0.001, 0.0001, 0.00001):
        if myDF is None:
            myDF = numLines(cc, pp)
        else:
            myDF = myDF.append (numLines(cc, pp))
myDF

,concur,lines,pB,pBlock
0,100,128,0.000968,0.00100
0,100,137,0.000074,0.00010
0,100,143,0.000010,0.00001
0,110,139,0.000999,0.00100
0,110,148,0.000088,0.00010
0,110,155,0.000009,0.00001
0,120,151,0.000806,0.00100
0,120,160,0.000076,0.00010
0,120,167,0.000009,0.00001
